In [50]:

import mysql.connector
import pandas as pd
conn = mysql.connector.connect(
    host = "database-1.cd2is2gsweff.ap-northeast-2.rds.amazonaws.com",
    port = "3306",
    user = "admin",
    password = "kim82458529",
    database = "amr_base"
)
cursor = conn.cursor(buffered=True)
cursor.execute("SELECT * FROM yousinsa")
columns = [i[0] for i in cursor.description]
df = pd.DataFrame(cursor.fetchall(), columns=columns)
cursor.close()
conn.close()



In [ ]:
df.head()

In [52]:
import ast
import pandas as pd
import re
def get_score(df, user_age, user_sex, middle_category):
    # 좋아요, 리뷰 만족도
    like_list = []
    for idx, data in df.iterrows():
        try:
            like_list.append(data["likes"] / data["review"])
        except ZeroDivisionError:
            like_list.append(0)
    likes_list = [like / max(like_list) for like in like_list]
    df['만족도'] = likes_list
    # 판매량
    df["total"] = df["sold"] * df["price_member"]
    df["total"] = df["total"] / max(df["total"])
    # 선호나이
    pd.set_option("mode.chained_assignment", None)
    df['age'] = df['age'].apply(lambda x: int(re.findall(r'\d+', str(x))[0]) if re.findall(r'\d+', str(x)) else 0)
    df["age"] = abs(df["age"] - user_age)
    df["age"] = df["age"] / max(df["age"])
    # 브랜드별 만족도를 최대 만족도로 나누어 정규화
    max_brand_satisfaction = df.groupby('brand')['만족도'].max()
    df['brand 만족도'] = df.apply(lambda row: row['만족도'] / max_brand_satisfaction.get(row['brand'], 1), axis=1)
    # sex -> 리스트화 후 input과 다르면 0
    # 총점 계산
    df["총점"] = df["만족도"] * 4 + df["total"] * 3 + df["brand 만족도"] * 2 + df["age"] * 1
    # 사용자 input 값에 따른 선별
    df.loc[~df['sex'].apply(lambda x: user_sex in x), '총점'] = 0
    df.loc[df["sort1"] != middle_category, '총점'] = 0
    new_df = df.copy()  # 원본 DataFrame을 변경하지 않도록 복사본 사용
    return new_df
# 예시 호출
result_df = get_score(df, 20, "남성", "후드 집업")
result_df

/tmp/ipykernel_18891/3557172059.py:24: RuntimeWarning: invalid value encountered in scalar divide
  df['brand 만족도'] = df.apply(lambda row: row['만족도'] / max_brand_satisfaction.get(row['brand'], 1), axis=1)


,ranking,link,name,brand,sex,sale,sort,sort1,likes,review,...,price_nomember,tag,number,sold,age,like_sex,만족도,total,brand 만족도,총점
0,0,https://www.musinsa.com/app/goods/1778404?loc=...,2WAY 스웻 후드 집업 (MELANGE GREY),toffee,"['남성', '여성']",13,아우터,후드 집업,218205,81445,...,45000,"후드, 집업후드, 트레이닝세트, 셋업, 세트, 후드티, 오버핏후드티, 무지, 그레이...",1778404,99541,0.05,남성,0.004311,0.694426,0.061007,2.272537
1,1,https://www.musinsa.com/app/goods/3777826?loc=...,LONELY/LOVELY WASHED HOODIE ZIP-UP_GRAY,nohant,['여성'],0,아우터,후드 집업,56,0,...,188000,"후드집업, 후드\집업, 후드집업추천, 오버핏, 집업후드, 론리러블리",3777826,0,1.00,0,0.000000,0.000000,0.000000,0.000000
2,2,https://www.musinsa.com/app/goods/3493145?loc=...,[SET] Very Beloa Hood Zip-up & Very Beloa Pant...,avan,['여성'],0,아우터,후드 집업,5040,428,...,130000,"후드집업, 후드, 집업후드, 스웻셔츠, 후디, 집업, 패키지",3493145,1335,0.05,여성,0.018947,0.021707,0.075809,0.000000
3,3,https://www.musinsa.com/app/goods/2897038?loc=...,폴라베어 와펜 후드집업 (JM5TZU764IV),jeep,['남성'],0,아우터,후드 집업,1689,36,...,139000,"아울렛, 후드티, 오버핏후드티",2897038,242,0.10,여성,0.075489,0.003380,0.226651,0.865400
4,4,https://www.musinsa.com/app/goods/2730700?loc=...,[SET] 크롭후디 OR 하프집업 셋업_멜란지,partimentowomen,['여성'],25,아우터,후드 집업,17096,905,...,86900,"스웻셔츠, 오버핏, 맨투맨, 스웨트셔츠, 집업, 후디, 후드집업",2730700,1193,0.20,여성,0.030395,0.013877,0.057593,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5665,85,https://www.musinsa.com/app/goods/320924?loc=g...,소가죽 첼시부츠 로빈 브라운,daytrip,['남성'],0,신발,부츠,16,1,...,89000,"신발, 워커, 첼시부츠",320924,0,1.00,0,0.025744,0.000000,1.000000,0.000000
5666,86,https://www.musinsa.com/app/goods/392776?loc=g...,4.7cm 카스타솔 미니멀 지퍼 앵클부츠 3335 [BLACK FG],firenzeatelier,['남성'],10,신발,부츠,176,15,...,205000,"앵글부츠, 블랙, 가죽부츠, 수제화부츠, 지퍼부츠, 앵클부츠",392776,0,1.00,0,0.018879,0.000000,0.278261,0.000000
5667,87,https://www.musinsa.com/app/goods/426885?loc=g...,[비브람] 군용워커 모티브 풀그레인 지퍼디테일 워커 9300 [BLACK FG],firenzeatelier,['남성'],0,신발,부츠,339,10,...,261000,"군용워커, 군화, 레이스업부츠, 블랙, 남자부츠, 수제화, 비브람",426885,0,1.00,0,0.054545,0.000000,0.803953,0.000000
5668,88,https://www.musinsa.com/app/goods/451852?loc=g...,여성 팀버랜드 6인치 프리미엄 부츠 / 10361-713,timberland,['여성'],0,신발,부츠,6084,414,...,268000,"부츠, 6인치, 6인치프리미엄, 팀버랜드우먼, 팀버랜드부츠, 6인치부츠, 시그니처,...",451852,263,0.05,여성,0.023645,0.014011,0.361962,0.000000


In [51]:
import ast
import pandas as pd
import re
def get_score(df, user_age, user_sex, middle_category):
    # 좋아요, 리뷰 만족도
    like_list = []
    for idx, data in df.iterrows():
        try:
            like_list.append(data["likes"] / data["review"])
        except ZeroDivisionError:
            like_list.append(0)
    likes_list = [like / max(like_list) for like in like_list]
    new_df = df.copy()  # 원본 DataFrame을 변경하지 않도록 복사본 사용
    
    new_df['만족도'] = likes_list
    # 매출
    new_df["total"] = df["sold"] * df["price_member"]
    # 매출 최대치 1로 평균화
    new_df["total"] = new_df["total"] / max(new_df["total"])
    # 선호나이
    #pd.set_option("mode.chained_assignment", None)
    new_df['age'] = df['age'].apply(lambda x: int(re.findall(r'\d+', str(x))[0]) if re.findall(r'\d+', str(x)) else 0)
    new_df["age"] = abs(df["age"] - user_age)
    new_df["age"] = df["age"] / max(df["age"])
    # 브랜드별 만족도를 최대 만족도로 나누어 정규화
    max_brand_satisfaction = df.groupby('brand')['만족도'].mean()
    new_df['brand 만족도'] = df.apply(lambda row: row['만족도'] / max_brand_satisfaction.get(row['brand'], 1), axis=1)
    # sex -> 리스트화 후 input과 다르면 0
    # 총점 계산
    new_df["총점"] = (new_df["만족도"] * 4 + new_df["total"] * 3 + new_df["brand 만족도"] * 2 + df["age"])
    # 사용자 input 값에 따른 선별
    new_df.loc[~new_df['sex'].apply(lambda x: user_sex in x), '총점'] = 0
    new_df.loc[new_df["sort1"] != middle_category, '총점'] = 0
    
    return new_df
    
    
    
    #new_df = new_df.sort_values(by="총점", ascending=False)

    #return list(new_df["link"][:6])
# 예시 호출
result_df = get_score(df, 20, "남성", "후드 집업")
#result_df

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [44]:
result_df["age"]

0       19
1        0
2       19
3       18
4       24
        ..
5665     0
5666     0
5667     0
5668    19
5669    24
Name: age, Length: 5670, dtype: int64

In [45]:
df["age"]

0       19~23,  남성
1                0
2       19~23,  여성
3          18,  여성
4       24~28,  여성
           ...    
5665             0
5666             0
5667             0
5668    19~23,  여성
5669    24~28,  남성
Name: age, Length: 5670, dtype: object

In [241]:
user_age = 20 #########유저 나이 들어갈곳
pd.set_option("mode.chained_assignment", None)
for a in range(len(df["age"])):
    try:
        df["age"][a] = list(map(int, df["age"][a].split("~")))
        df["age"][a] = int((df["age"][a][0] + df["age"][a][1])/2)
    except:
        if df["age"][a] != [0]:
            df["age"][a] = df["age"][a][0]
        else:
            df["age"][a] = 0
        
df["age"] = abs(df["age"] - user_age)
df["age"] = df["age"] / max(df["age"])



In [136]:
df["total"] = df["sold"] * df["price_nomember"]
df["total"] = df["total"] / max(df["total"])
(df["total"])


0       0.529478
1       0.000000
2       0.020514
3       0.003976
4       0.012254
          ...   
5672    0.000000
5673    0.000000
5674    0.000000
5675    0.008332
5676    0.002313
Name: total, Length: 5677, dtype: float64

In [18]:
new_df1 = df.sort_values(by="총점",ascending=False)

In [19]:
new_df1

,ranking,link,name,brand,sex,sale,sort,sort1,likes,review,...,price_nomember,tag,number,sold,age,like_sex,만족도,total,brand 만족도,총점
20,20,https://www.musinsa.com/app/goods/2700860?loc=...,빅로고 후드 집업 (JN5TZU002BK),jeep,['남성'],63,아우터,후드 집업,621,3,...,109000,"집업, 오버핏후드, 후드티, 후드, 후디, 후드자켓, 후드집업, 로고, 블랙, 머스트헤브",2700860,31,1.00,여성,0.333065,0.000249,1.000000,4.333006
68,68,https://www.musinsa.com/app/goods/3512114?loc=...,BLACK HOLE 후드집업(STJSTD-0062),1989standard,['남성'],0,아우터,후드 집업,415,2,...,98000,"후드집업, 남자후드집업, 여자후드집업, 집업, 집업후드, 오버핏후드집업, 23FW컬렉션",3512114,44,0.95,여성,0.333870,0.000432,1.000000,4.286774
34,34,https://www.musinsa.com/app/goods/3777854?loc=...,LONELY/LOVELY WASHED HOODIE ZIP-UP_CHARCOAL,nohant,['남성'],0,아우터,후드 집업,114,1,...,188000,"후드집업, 후드집업추천, 후드\집업, 집업후드, 론리러블리",3777854,20,0.95,남성,0.183427,0.000748,1.000000,3.685953
30,30,https://www.musinsa.com/app/goods/3563124?loc=...,엔비엘 오버핏 후드집업 (오트밀),tailorstudio,['남성'],54,아우터,후드 집업,1255,15,...,129000,"후드집업, 후드집업추천, 로고후드집업, 자수로고후드집업, 오버핏후드집업, 남여공용,...",3563124,89,1.00,여성,0.134621,0.001060,1.000000,3.541661
59,59,https://www.musinsa.com/app/goods/3613148?loc=...,러닝 피그먼트 후드 집업 딥그레이 SHOT6742,ezkaton,['남성'],0,아우터,후드 집업,2867,42,...,111000,"후드티셔츠, 오버핏후드, 오버핏후드티, 후드티추천, 후드티, 그래픽후드, 오버핏",3613148,265,0.95,남성,0.109834,0.003677,1.000000,3.400367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1926,36,https://www.musinsa.com/app/goods/3259951?loc=...,S23MUFWC72 나일론 와치캡 Black,snowpeakapparel,['남성'],20,모자,비니,111,23,...,49000,"로고, 아웃도어, 와치캡, 비니, 모자추천, 스노우피크×류승범, 캠핑",3259951,96,0.95,남성,0.007765,0.000748,0.281992,0.000000
1925,35,https://www.musinsa.com/app/goods/1886377?loc=...,CL Summer Stickcap Mustard Yellow,ramolin,['남성'],15,모자,비니,574,190,...,48000,"비니모자, 여름비니, 코튼비니, 숏비니, 여름모자, 비니",1886377,192,1.00,남성,0.004861,0.001574,1.000000,0.000000
1924,34,https://www.musinsa.com/app/goods/3616500?loc=...,투 톤 니트 비니 (차콜),heim,"['남성', '여성']",5,모자,비니,853,21,...,49000,no data,3616500,99,0.95,남성,0.065356,0.000731,1.000000,0.000000
1923,33,https://www.musinsa.com/app/goods/3023694?loc=...,Earmuffs Beanie Ivory,thegreatest,['여성'],0,모자,비니,734,5,...,49000,"겨울잡화, 니트비니, 남자비니, 비니모자, 비니추천, 여자비니, 귀도리모자",3023694,29,0.95,여성,0.236203,0.000282,1.000000,0.000000


In [165]:
import ast
cate = "후드 집업"
df["가격"] = df["price_member"]
df["sex"] = df.apply(lambda x: ast.literal_eval(x.sex), axis=1)
df.loc[df['sex'].apply(lambda x: '남성' in x), 'price_member'] = 0
#df["남성" in df["sort1"]].가격


In [220]:
#df["sex"].isin(["남성","여성"])
df["sex"]

0       [남성, 여성]
1           [여성]
2           [여성]
3           [남성]
4           [여성]
          ...   
5672        [남성]
5673        [남성]
5674        [남성]
5675        [여성]
5676        [남성]
Name: sex, Length: 5677, dtype: object

,ranking,link,name,brand,sex,sale,sort,sort1,likes,review,price_member,price_nomember,tag,number,sold,age,total,가격


In [233]:
df.loc[~df['sex'].apply(lambda x: '남성' in x), 'price_member'] = 0
df

SyntaxError: invalid syntax (2737650753.py, line 1)

In [234]:
df["sort1"]

0       후드 집업
1       후드 집업
2       후드 집업
3       후드 집업
4       후드 집업
        ...  
5672       부츠
5673       부츠
5674       부츠
5675       부츠
5676       부츠
Name: sort1, Length: 5677, dtype: object